In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/new_master_dataset.csv")
df

Mounted at /content/drive


,mag,path,filename,class,slide_id,tumor_type
0,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-001.png,benign,22549CD,A
1,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-002.png,benign,22549CD,A
2,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-003.png,benign,22549CD,A
3,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-004.png,benign,22549CD,A
4,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-005.png,benign,22549CD,A
...,...,...,...,...,...,...
7904,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-031.png,malignant,15704,PC
7905,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-032.png,malignant,15704,PC
7906,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-033.png,malignant,15704,PC
7907,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-034.png,malignant,15704,PC


In [2]:
groupd_df = df.groupby("tumor_type")

# You can then perform various operations on the grouped data, such as getting the count of each tumor type
tumor_type_counts = groupd_df.size()
tumor_type_counts

tumor_type
A      444
DC    3451
F     1014
LC     626
MC     792
PC     560
PT     453
TA     569
dtype: int64

In [3]:
# df_train_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/train_df_100.csv")
# df_test_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/test_df_100.csv")
# df_val_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/val_df_100.csv")

In [5]:
!pip install timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from PIL import Image
import os
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification
import timm

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Define constants
data_dirs = ["TA", "PT", "PC", "MC", "LC", "F", "DC", "A"]
data_root = "/content/drive/MyDrive/Breast Cancer Project/IW/400"  # Replace with the root directory of your data
train_split = 0.7

# Create a list to store the paths and labels of all images
all_data = []

# Populate the list with paths and labels
for label, folder in enumerate(data_dirs):
    folder_path = os.path.join(data_root, folder)
    image_files = os.listdir(folder_path)
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        all_data.append((image_path, label))

# Split data into training and testing sets
train_data, test_data = train_test_split(all_data, train_size=train_split, shuffle=True, random_state=42)

# Define custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        img = Image.open(img_path).convert('RGB')  # Open image and convert to RGB mode
        if self.transform:
            img = self.transform(img)
        label_tensor = torch.tensor(label, dtype=torch.long)  # Convert label to tensor
        return img, label_tensor

# Image preprocessing with augmentation for training
train_transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.RandomRotation(90),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

# Image preprocessing without augmentation for testing and validation
test_val_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor()
])

# Create custom datasets
train_dataset = CustomDataset(train_data, transform=train_transform)
test_dataset = CustomDataset(test_data, transform=test_val_transform)

# DataLoaders for batching and shuffling
batch_size = 30  # Define the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

import timm

# Create model
model = timm.create_model('tf_efficientnetv2_s.in21k', pretrained=True)
model.to(device)  # Move model to GPU

# Define optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Using device: cuda


Epoch 1/10, Train Loss: 5.2573, Train Accuracy: 38.23%, Val Loss: 4.7614, Val Accuracy: 38.64%


Epoch 2/10, Train Loss: 1.7352, Train Accuracy: 65.31%, Val Loss: 3.0764, Val Accuracy: 51.10%


Epoch 3/10, Train Loss: 0.9896, Train Accuracy: 75.12%, Val Loss: 2.3748, Val Accuracy: 61.90%


Epoch 4/10, Train Loss: 0.5697, Train Accuracy: 82.50%, Val Loss: 1.6224, Val Accuracy: 68.32%


...

Epoch 6/10, Train Loss: 0.2802, Train Accuracy: 91.52%, Val Loss: 0.9967, Val Accuracy: 77.11%


Epoch 7/10, Train Loss: 0.2369, Train Accuracy: 91.84%, Val Loss: 0.9521, Val Accuracy: 79.67%


Epoch 8/10, Train Loss: 0.1889, Train Accuracy: 93.25%, Val Loss: 0.8339, Val Accuracy: 81.87%


Epoch 9/10, Train Loss: 0.1368, Train Accuracy: 95.45%, Val Loss: 0.7107, Val Accuracy: 85.16%


Epoch 10/10, Train Loss: 0.1334, Train Accuracy: 95.68%, Val Loss: 0.6627, Val Accuracy: 84.80%
Test Accuracy: 84.80%


In [6]:
import numpy as np
from sklearn.metrics import classification_report

# Evaluate the model on the test set
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Print classification report with three digits
print(classification_report(true_labels, predictions, target_names=data_dirs, digits=3))

              precision    recall  f1-score   support

          TA      0.725     0.935     0.817        31
          PT      0.806     0.967     0.879        30
          PC      0.947     0.735     0.828        49
          MC      0.870     0.769     0.816        52
          LC      0.641     0.932     0.759        44
           F      0.750     0.887     0.813        71
          DC      0.946     0.823     0.880       232
           A      0.944     0.919     0.932        37

    accuracy                          0.848       546
   macro avg      0.829     0.871     0.840       546
weighted avg      0.868     0.848     0.851       546



In [7]:
# Train the model
num_epochs = 20
for epoch in range(10, num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 11/20, Train Loss: 0.0969, Train Accuracy: 96.78%, Val Loss: 0.6564, Val Accuracy: 83.52%


Epoch 12/20, Train Loss: 0.0895, Train Accuracy: 97.25%, Val Loss: 0.5801, Val Accuracy: 85.35%


Epoch 13/20, Train Loss: 0.0728, Train Accuracy: 97.17%, Val Loss: 0.5817, Val Accuracy: 85.35%


Epoch 14/20, Train Loss: 0.0658, Train Accuracy: 97.49%, Val Loss: 0.4565, Val Accuracy: 87.55%


Epoch 15/20, Train Loss: 0.0824, Train Accuracy: 97.25%, Val Loss: 0.5386, Val Accuracy: 84.80%


Epoch 16/20, Train Loss: 0.0723, Train Accuracy: 97.25%, Val Loss: 0.4505, Val Accuracy: 88.28%


Epoch 17/20, Train Loss: 0.0665, Train Accuracy: 97.72%, Val Loss: 0.5074, Val Accuracy: 87.00%


Epoch 18/20, Train Loss: 0.0654, Train Accuracy: 97.41%, Val Loss: 0.4787, Val Accuracy: 87.73%


Epoch 19/20, Train Loss: 0.0696, Train Accuracy: 97.41%, Val Loss: 0.5052, Val Accuracy: 85.90%


Epoch 20/20, Train Loss: 0.0663, Train Accuracy: 97.72%, Val Loss: 0.5115, Val Accuracy: 85.35%
Epoch 00020: reducing learning rate of group 0 to 1.0000e-05.
Test Accuracy: 85.35%


In [8]:
import numpy as np
from sklearn.metrics import classification_report

# Evaluate the model on the test set
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Print classification report with three digits
print(classification_report(true_labels, predictions, target_names=data_dirs, digits=3))

              precision    recall  f1-score   support

          TA      0.829     0.935     0.879        31
          PT      0.824     0.933     0.875        30
          PC      0.891     0.837     0.863        49
          MC      0.772     0.846     0.807        52
          LC      0.683     0.636     0.659        44
           F      0.774     0.915     0.839        71
          DC      0.919     0.875     0.896       232
           A      1.000     0.757     0.862        37

    accuracy                          0.853       546
   macro avg      0.836     0.842     0.835       546
weighted avg      0.860     0.853     0.854       546



In [9]:
# Train the model
num_epochs = 30
for epoch in range(20, num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 21/30, Train Loss: 0.0476, Train Accuracy: 98.51%, Val Loss: 0.4606, Val Accuracy: 87.73%


Epoch 22/30, Train Loss: 0.0350, Train Accuracy: 98.98%, Val Loss: 0.4728, Val Accuracy: 85.90%


Epoch 23/30, Train Loss: 0.0436, Train Accuracy: 98.12%, Val Loss: 0.4570, Val Accuracy: 87.55%


Epoch 24/30, Train Loss: 0.0401, Train Accuracy: 98.35%, Val Loss: 0.4407, Val Accuracy: 87.55%


Epoch 25/30, Train Loss: 0.0309, Train Accuracy: 98.51%, Val Loss: 0.4281, Val Accuracy: 87.91%


Epoch 26/30, Train Loss: 0.0328, Train Accuracy: 98.82%, Val Loss: 0.4146, Val Accuracy: 89.19%


Epoch 27/30, Train Loss: 0.0371, Train Accuracy: 98.67%, Val Loss: 0.4281, Val Accuracy: 88.28%


Epoch 28/30, Train Loss: 0.0319, Train Accuracy: 98.51%, Val Loss: 0.4457, Val Accuracy: 88.10%


Epoch 29/30, Train Loss: 0.0316, Train Accuracy: 98.98%, Val Loss: 0.4111, Val Accuracy: 88.64%


Epoch 30/30, Train Loss: 0.0287, Train Accuracy: 98.74%, Val Loss: 0.4114, Val Accuracy: 89.01%
Test Accuracy: 89.01%


In [10]:
import numpy as np
from sklearn.metrics import classification_report

# Evaluate the model on the test set
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert predictions and true labels to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Print classification report with three digits
print(classification_report(true_labels, predictions, target_names=data_dirs, digits=3))

              precision    recall  f1-score   support

          TA      0.789     0.968     0.870        31
          PT      0.875     0.933     0.903        30
          PC      0.976     0.816     0.889        49
          MC      0.852     0.885     0.868        52
          LC      0.717     0.750     0.733        44
           F      0.859     0.944     0.899        71
          DC      0.934     0.914     0.924       232
           A      1.000     0.811     0.896        37

    accuracy                          0.890       546
   macro avg      0.875     0.878     0.873       546
weighted avg      0.896     0.890     0.891       546

